Using this credit card fraud dataset develop an algorithm to predict fraud. Prioritize correctly finding fraud rather than correctly labeling non-fraudulent transactions.



https://www.kaggle.com/dalpozz/creditcardfraud

In [38]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline


In [39]:
credit_card  = pd.read_csv("C:/Users/boyerm/Documents/Thinkful/creditcard.csv")

In [40]:
credit_card.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [41]:
# Define the training and test sizes.
trainsize = int(credit_card.shape[0] / 2)
credit_card_train = credit_card.iloc[:trainsize, :].copy()
credit_card_test = credit_card.iloc[trainsize:, :].copy()

In [42]:
data_train = credit_card_train.drop(['Class'],1)
target_train = credit_card_train.Class

data_test = credit_card_test.drop(['Class'],1)
target_test = credit_card_test.Class

## Model 1 - Naive Bayes

In [6]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(data_train, target_train)

# Classify, storing the result in a new variable.
y_pred_train = bnb.predict(data_train)
y_pred_test = bnb.predict(data_test)

# Display our results.
print("Train:Number of mislabeled points out of a total {} points : {}".format(
    data_train.shape[0],
    (target_train != y_pred_train).sum()
))
print("Test:Number of mislabeled points out of a total {} points : {}".format(
    data_test.shape[0],
    (target_test != y_pred_test).sum()
))

Train:Number of mislabeled points out of a total 142403 points : 128
Test:Number of mislabeled points out of a total 142404 points : 104


In [7]:
# this seems good BUT....

In [8]:
pd.crosstab(target_train, y_pred_train)

col_0,0,1
Class,,
0,142085,49
1,79,190


In [9]:
# Train: 79/269 = 29% error

In [10]:
pd.crosstab(target_test, y_pred_test)

col_0,0,1
Class,,
0,142160,21
1,83,140


In [13]:
# Test: 83/223 = 37% error!!

turns out naive bayes is not good w/class imbalance

## Model 2: random forest

In [14]:
# random forest
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()

cross_val_score(rfc, data_train, target_train, cv=10)

array([ 0.86524823,  0.99915736,  0.99831473,  0.9992978 ,  0.9997191 ,
        0.9994382 ,  0.99950843,  0.99950843,  0.99929775,  0.99894656])

In [15]:
rfc.fit(data_train, target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [16]:
# Classify, storing the result in a new variable.
y_pred_test2= rfc.predict(data_test)
y_pred_train2 = rfc.predict(data_train)

In [17]:
# Display our results.
print("Train: Number of mislabeled points out of a total {} points : {}".format(
    data_train.shape[0],
    (target_train != y_pred_train2).sum()
))
print("Test: Number of mislabeled points out of a total {} points : {}".format(
    data_test.shape[0],
    (target_test != y_pred_test2).sum()
))

Train: Number of mislabeled points out of a total 142403 points : 18
Test: Number of mislabeled points out of a total 142404 points : 115


In [18]:
pd.crosstab(target_train, y_pred_train2)

col_0,0,1
Class,,
0,142134,0
1,18,251


In [19]:
# this works well on the training db - 5.2% error

In [20]:
pd.crosstab(target_test, y_pred_test2)

col_0,0,1
Class,,
0,142138,43
1,72,151


In [21]:
# for the test db -  72 /223 - 32%

pretty good!

## logistic regression

In [22]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=0.2)

In [23]:
lr.fit(data_train, target_train)

LogisticRegression(C=0.2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
y_pred_train3 = lr.predict(data_train)
y_pred_test3 = lr.predict(data_test)

In [25]:
pd.crosstab(target_train, y_pred_train3)

col_0,0,1
Class,,
0,142093,41
1,138,131


In [26]:
# not good

In [34]:
pd.crosstab(target_test, y_pred_test3)

col_0,0,1
Class,,
0,142175,6
1,180,43


In [28]:
# AWFUl!

## Support Vector Machines

In [69]:
from sklearn.svm import SVC

In [70]:
svm = SVC()
svm.fit(data_train, target_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [71]:
y_pred_train4 = svm.predict(data_train)
y_pred_test4 = svm.predict(data_test)

In [72]:
pd.crosstab(target_train, y_pred_train4)

col_0,0,1
Class,,
0,142134,0
1,33,236


In [73]:
pd.crosstab(target_test, y_pred_test4)

col_0,0
Class,
0,142181
1,223


In [ ]:
# this is weird. it didn't predict any fraud in the test set?

## Try Gradient Boosting:

In [43]:
# We'll make 500 iterations, use 2-deep trees, and set our loss function.
params = {'n_estimators': 500,
          'max_depth': 2,
          'loss': 'deviance'}

# Initialize and fit the model.
clf = ensemble.GradientBoostingClassifier(**params)
clf.fit(data_train, target_train)

y_pred_train5 = clf.predict(data_train)
y_pred_test5 = clf.predict(data_test)




In [61]:
# Accuracy tables.
table_train = pd.crosstab(target_train, y_pred_train5, margins = True )
print(table_train)

col_0       0    1     All
Class                     
0      142111   23  142134
1          87  182     269
All    142198  205  142403


In [ ]:
#87/269 = 32% missed

In [62]:
table_test = pd.crosstab(target_test, y_pred_test5, margins = True )
print(table_test)

col_0       0    1     All
Class                     
0      142150   31  142181
1         106  117     223
All    142256  148  142404


In [ ]:
# 47.5% error for fraud

In [68]:
train_tI_errors = table_train.loc[0,1] / table_train.loc['All','All'] 
train_tII_errors = table_train.loc[1,0] / table_train.loc['All','All']

test_tI_errors = table_test.loc[0,1]/table_test.loc['All','All']
test_tII_errors = table_test.loc[1,0]/table_test.loc['All','All']

print((
    'Training set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}\n\n'
    'Test set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}'
).format(train_tI_errors, train_tII_errors, test_tI_errors, test_tII_errors))

Training set accuracy:
Percent Type I errors: 0.0001615134512615605
Percent Type II errors: 0.0006109421852067723

Test set accuracy:
Percent Type I errors: 0.00021769051431139576
Percent Type II errors: 0.0007443611134518693


In [ ]:
# moving On :)